## Dashboard Creation and Automated Reporting Instruction Manual

### Required Application Downloads

### Grab and Open Files

Two cells deep do exactly what i want it do do

In [140]:
#Library Importation
import openpyxl
from openpyxl import load_workbook
import pandas as pd
import numpy as np

#Loading Necessary Workbooks
wb1 = load_workbook ("StaffPerformanceOverviewCR06-23-23 test copy.xlsx")
wb2 = load_workbook("StaffPerformanceOverviewNH06-23-23 test copy.xlsx")
ws1 = wb1.active
ws2 = wb2.active

#Appending Data To One Workbook
for row in ws2.iter_rows(min_row=2, values_only=True): 
    ws1.append(row)

wb1.save("StaffPerformanceOverviewtest-06-23 test copy.xlsx")

#reload new workbook that was just saved
wb = load_workbook("StaffPerformanceOverviewtest-06-23 test copy.xlsx")
ws = wb.active

#Remove Row to consolidate data
start_row = 12
num_rows_to_remove = 1
ws.delete_rows(start_row, num_rows_to_remove)

# Define the column letter you want to update (e.g., column A)
column_letter = 'E'

# Define the value to be replaced and its corresponding replacement
old = "NA"
new = "0"

# Get the column index from the column letter
column_index = ord(column_letter) - ord('A') + 1

# Loop through the cells in the selected column and replace the old value with the new value
for row in ws.iter_rows(min_row=3, min_col=column_index, max_col=column_index):
    cell = row[0]
    if cell.value == old:
        cell.value = new

from openpyxl.styles import NamedStyle

# Create named style for the whole number format
whole_number_style = NamedStyle(name="whole_number", number_format="0")

# Columns that need to be formatted as a whole number (example: columns A, B, and D)
columns_to_format = ['B', 'C', 'D']  # Replace with the letters of the columns you want to format

# Apply the named style to each cell in the selected columns
for column_letter in columns_to_format:
    for cell in ws[column_letter]:
        cell.style = whole_number_style

Decimal_Number_Style = NamedStyle(name="decimal", number_format="#,##0.00")

columns_to_format = ['E', 'F', 'G', 'H','I','J','K','L','M','N','O']

for column_letter in columns_to_format:
    for cell in ws[column_letter]:
        cell.style = Decimal_Number_Style


wb.save("StaffPerformanceOverviewtest-06-23 test copy.xlsx")

In [141]:
from openpyxl import load_workbook

# Load the existing Excel workbook
wb = load_workbook("StaffPerformanceOverviewtest-06-23 test copy.xlsx")

# Select the worksheet you want to update (assuming it's the first sheet)
ws = wb.active

# Find the last row in the worksheet
last_row = ws.max_row

# Create a dictionary to store the rows of employees with multiple entries
multiple_entries_rows = {}

# Iterate through the rows and collect the rows of employees with multiple entries
for row in range(3, last_row + 1):
    employee_id = ws.cell(row=row, column=1).value  # Assuming 'EmployeeID' is in column A
    if employee_id in multiple_entries_rows:
        multiple_entries_rows[employee_id].append([cell.value for cell in ws[row]])
    else:
        multiple_entries_rows[employee_id] = [[cell.value for cell in ws[row]]]

# Clear the existing data in the worksheet
ws.delete_rows(3, last_row)

# Write the rows of employees with single entries at the top of the worksheet
starting_row_for_multiple_entries = 11  # Adjust this variable for different reports
for employee_id, rows in multiple_entries_rows.items():
    if len(rows) == 1:
        ws.append(rows[0])  # Append the single row for employees with single entries

# Write the rows of employees with multiple entries at the bottom of the worksheet starting at the specified row
for employee_id, rows in multiple_entries_rows.items():
    if len(rows) > 1:
        for row_data in rows:
            ws.insert_rows(starting_row_for_multiple_entries, 1)  # Insert a row at the specified row for employees with multiple entries
            for col_index, cell_value in enumerate(row_data):
                ws.cell(row=starting_row_for_multiple_entries, column=col_index + 1, value=cell_value)
            starting_row_for_multiple_entries += 1  # Increment the starting row for the next employee

# Save the updated workbook
wb.save("StaffPerformanceOverviewtest-06-23 test copy.xlsx")



***

In [142]:
import pandas as pd
from openpyxl import load_workbook

# Load the existing Excel workbook
wb = load_workbook("StaffPerformanceOverviewtest-06-23 test copy.xlsx")

# Select the worksheet you want to update (assuming it's the first sheet)
ws = wb.active

# Read the data from the worksheet into a Pandas DataFrame
data_range = ws['A11:O18']  # Assuming data is in columns A to K starting from row 2
data = [[cell.value for cell in row] for row in data_range]

# Create a DataFrame with all the data
df = pd.DataFrame(data, columns=['EmployeeID', 'Data1', 'Data2', 'Data3', 'Data4', 'Data5', 'Data6', 'Data7', 
                                  'Data8', 'Data9', 'Data10', 'Data11', 'Data12', 'Data13', 'Data14'])

# Get the union of EmployeeIDs
all_employee_ids = set(df['EmployeeID'])

# Create a DataFrame with all EmployeeIDs
df_all_employees = pd.DataFrame({'EmployeeID': list(all_employee_ids)})

# Merge the original data with the DataFrame containing all EmployeeIDs (outer merge)
merged_data = pd.merge(df_all_employees, df, on='EmployeeID', how='left')

# Fill NaN values with 0
merged_data.fillna(0, inplace=True)

# Convert 'Data4', 'Data13', and 'Data14' columns to numeric type (ignore errors)
merged_data['Data4'] = pd.to_numeric(merged_data['Data4'], errors='coerce')
merged_data['Data13'] = pd.to_numeric(merged_data['Data13'], errors='coerce')
merged_data['Data14'] = pd.to_numeric(merged_data['Data14'], errors='coerce')

# Consolidate data based on EmployeeID and calculate the sum for each of the 14 columns
consolidated_data = merged_data.groupby('EmployeeID', as_index=False).agg({
    'Data1': 'sum',
    'Data2': 'sum',
    'Data3': 'sum',
    'Data4': 'sum',
    'Data5': 'sum',
    'Data6': 'sum',
    'Data7': 'sum',
    'Data8': 'sum',
    'Data9': 'sum',
    'Data10': 'sum',
    'Data11': 'sum',
    'Data12': 'sum',
    'Data13': 'sum',
    'Data14': 'sum'})

# Calculate the average for Data4, Data13, and Data14 columns for each individual employee
consolidated_data['Data4'] = consolidated_data['Data4'] / 2
consolidated_data['Data13'] = consolidated_data['Data13'] / 2
consolidated_data['Data14'] = consolidated_data['Data14'] / 2

# Determine the starting row for placing the consolidated data
start_row = 20  # Assuming you want to start placing the consolidated data in row 20

# Put the consolidated data back into the Excel worksheet
for row_index, row_data in enumerate(consolidated_data.values):
    for col_index, cell_value in enumerate(row_data):
        cell = ws.cell(row=start_row + row_index, column=col_index + 1)
        cell.value = cell_value

# Save the updated workbook
wb.save("StaffPerformanceOverviewtest-06-23 test copy.xlsx")



pause

In [124]:
import pandas as pd
from openpyxl import load_workbook

# Load the existing Excel workbook
wb = load_workbook("StaffPerformanceOverviewtest-06-23 test copy.xlsx")

# Select the worksheet you want to update (assuming it's the first sheet)
ws = wb.active

# Read the data from the worksheet into a Pandas DataFrame
data_range = ws['A3:O18']  # Assuming data is in columns A to K starting from row 2
data = [[cell.value for cell in row] for row in data_range]

# Create a DataFrame with all the data
df = pd.DataFrame(data, columns=['EmployeeID', 'Data1', 'Data2', 'Data3', 'Data4', 'Data5', 'Data6', 'Data7', 
                                  'Data8', 'Data9', 'Data10', 'Data11', 'Data12', 'Data13', 'Data14'])

# Convert 'Data4', 'Data13', and 'Data14' to numeric (ignore errors)
df['Data4'] = pd.to_numeric(df['Data4'], errors='coerce')
df['Data13'] = pd.to_numeric(df['Data13'], errors='coerce')
df['Data14'] = pd.to_numeric(df['Data14'], errors='coerce')

# Get the union of EmployeeIDs
all_employee_ids = set(df['EmployeeID'])

# Create a DataFrame with all EmployeeIDs
df_all_employees = pd.DataFrame({'EmployeeID': list(all_employee_ids)})

# Merge the original data with the DataFrame containing all EmployeeIDs (outer merge)
merged_data = pd.merge(df_all_employees, df, on='EmployeeID', how='left')

# Fill NaN values with 0
merged_data.fillna(0, inplace=True)

# Consolidate data based on EmployeeID and calculate the sum for each of the 8 columns
sum_columns = ['Data1', 'Data2', 'Data3', 'Data5', 'Data6', 'Data7', 'Data8', 'Data9', 'Data10', 'Data11', 'Data12']
consolidated_data = merged_data.groupby('EmployeeID', as_index=False)[sum_columns].sum()

# Calculate the average for Data4, Data13, and Data14 columns and divide by 2
consolidated_data['Data4'] = merged_data['Data4'].groupby(merged_data['EmployeeID']).transform('mean') / 2
consolidated_data['Data13'] = merged_data['Data13'].groupby(merged_data['EmployeeID']).transform('mean') / 2
consolidated_data['Data14'] = merged_data['Data14'].groupby(merged_data['EmployeeID']).transform('mean') / 2

# Determine the starting row for placing the consolidated data
start_row = 20  # Assuming you want to start placing the consolidated data in row 20

# Put the consolidated data back into the Excel worksheet
for row_index, row_data in enumerate(consolidated_data.values):
    for col_index, cell_value in enumerate(row_data):
        cell = ws.cell(row=start_row + row_index, column=col_index + 1)
        cell.value = cell_value

# Save the updated workbook
wb.save("StaffPerformanceOverviewtest-06-23 test copy.xlsx")



This code below does exactly what i want right after the code above

In [71]:
import pandas as pd
from openpyxl import load_workbook

# Load the existing Excel workbook
wb = load_workbook("StaffPerformanceOverviewtest-06-23 test copy.xlsx")

# Select the worksheet you want to update (assuming it's the first sheet)
ws = wb.active

# Read the data from the worksheet into a Pandas DataFrame
data_range_location1 = ws['A3:O11']  # Assuming data for location 1 is in columns A to K starting from row 2
data_location1 = [[cell.value for cell in row] for row in data_range_location1]

data_range_location2 = ws['A12:O18']  # Assuming data for location 2 is in columns A to K starting from row 2
data_location2 = [[cell.value for cell in row] for row in data_range_location2]

# Create separate DataFrames for location 1 and location 2
df_location1 = pd.DataFrame(data_location1, columns=['EmployeeID', 'Data1', 'Data2', 'Data3', 'Data4', 'Data5', 'Data6', 'Data7', 
                                                                                            'Data8', 'Data9', 'Data10', 'Data11', 'Data12', 'Data13', 'Data14'])
df_location2 = pd.DataFrame(data_location2, columns=['EmployeeID', 'Data1', 'Data2', 'Data3', 'Data4', 'Data5', 'Data6', 'Data7',
                                                                                             'Data8', 'Data9', 'Data10', 'Data11', 'Data12', 'Data13', 'Data14'])

# Get the union of EmployeeIDs from both data frames
all_employee_ids = set(df_location1['EmployeeID']).union(set(df_location2['EmployeeID']))

# Create DataFrames with all EmployeeIDs for both locations
df_all_location1 = pd.DataFrame({'EmployeeID': list(all_employee_ids)})
df_all_location2 = pd.DataFrame({'EmployeeID': list(all_employee_ids)})

# Merge the original data with the DataFrames containing all EmployeeIDs (outer merge)
merged_location1 = pd.merge(df_all_location1, df_location1, on='EmployeeID', how='left')
merged_location2 = pd.merge(df_all_location2, df_location2, on='EmployeeID', how='left')

# Fill NaN values with 0
merged_location1.fillna(0, inplace=True)
merged_location2.fillna(0, inplace=True)

# Consolidate data based on EmployeeID and calculate the sum for each location
consolidated_data_location1 = merged_location1.groupby('EmployeeID', as_index=False).sum()
consolidated_data_location2 = merged_location2.groupby('EmployeeID', as_index=False).sum()

# Merge the consolidated data for employees working at both locations (outer merge)
consolidated_data_both_locations = pd.merge(consolidated_data_location1, consolidated_data_location2, on='EmployeeID', suffixes=('_loc1', '_loc2'), how='outer')

# Calculate the final values for the Data1, Data2, Data3 columns (sum for all)
consolidated_data_both_locations['Data1'] = consolidated_data_both_locations['Data1_loc1'] + consolidated_data_both_locations['Data1_loc2']
consolidated_data_both_locations['Data2'] = consolidated_data_both_locations['Data2_loc1'] + consolidated_data_both_locations['Data2_loc2']
consolidated_data_both_locations['Data3'] = consolidated_data_both_locations['Data3_loc1'] + consolidated_data_both_locations['Data3_loc2']
consolidated_data_both_locations['Data5'] = consolidated_data_both_locations['Data5_loc1'] + consolidated_data_both_locations['Data5_loc2']
consolidated_data_both_locations['Data6'] = consolidated_data_both_locations['Data6_loc1'] + consolidated_data_both_locations['Data6_loc2']
consolidated_data_both_locations['Data7'] = consolidated_data_both_locations['Data7_loc1'] + consolidated_data_both_locations['Data7_loc2']
consolidated_data_both_locations['Data8'] = consolidated_data_both_locations['Data8_loc1'] + consolidated_data_both_locations['Data8_loc2']
consolidated_data_both_locations['Data9'] = consolidated_data_both_locations['Data9_loc1'] + consolidated_data_both_locations['Data9_loc2']
consolidated_data_both_locations['Data10'] = consolidated_data_both_locations['Data10_loc1'] + consolidated_data_both_locations['Data10_loc2']
consolidated_data_both_locations['Data11'] = consolidated_data_both_locations['Data11_loc1'] + consolidated_data_both_locations['Data11_loc2']
consolidated_data_both_locations['Data12'] = consolidated_data_both_locations['Data12_loc1'] + consolidated_data_both_locations['Data12_loc2']

# Calculate the average for Data4 to Data6 columns
consolidated_data_both_locations['Data4'] = (consolidated_data_both_locations['Data4_loc1'] + consolidated_data_both_locations['Data4_loc2']) / 2
consolidated_data_both_locations['Data13'] = (consolidated_data_both_locations['Data13_loc1'] + consolidated_data_both_locations['Data13_loc2']) / 2
consolidated_data_both_locations['Data14'] = (consolidated_data_both_locations['Data14_loc1'] + consolidated_data_both_locations['Data14_loc2']) / 2

# Remove the unnecessary columns from the consolidated DataFrame
consolidated_data_both_locations.drop(columns=[
    'Data1_loc1', 'Data1_loc2', 'Data2_loc1', 'Data2_loc2', 'Data3_loc1', 'Data3_loc2',
    'Data4_loc1', 'Data4_loc2', 'Data5_loc1', 'Data5_loc2', 'Data6_loc1', 'Data6_loc2',
    'Data7_loc1', 'Data7_loc2', 'Data8_loc1', 'Data8_loc2', 'Data9_loc1', 'Data9_loc2', 
    'Data10_loc1', 'Data10_loc2', 'Data11_loc1', 'Data11_loc2', 'Data12_loc1', 'Data12_loc2',
     'Data13_loc1', 'Data13_loc2',  'Data14_loc1', 'Data14_loc2'], inplace=True)

# Convert the consolidated data to a list
consolidated_data_list = consolidated_data_both_locations.values.tolist()

# Determine the starting row for placing the consolidated data
start_row = 20  # Assuming you want to start placing the consolidated data in row 20

# Put the consolidated data back into the Excel worksheet
for row_index, row_data in enumerate(consolidated_data_list):
    for col_index, cell_value in enumerate(row_data):
        cell = ws.cell(row=start_row + row_index, column=col_index + 1)
        cell.value = cell_value

# Save the updated workbook
wb.save("StaffPerformanceOverviewtest-06-23 test copy.xlsx")



KeyError: 'Data12'

***


***

In [23]:
import pandas as pd
from openpyxl import load_workbook

# Load the existing Excel workbook
wb = load_workbook("StaffPerformanceOverviewtest-06-23 test copy.xlsx")

# Select the worksheet you want to update (assuming it's the first sheet)
ws = wb.active

# Read the data from the worksheet into a Pandas DataFrame
data_range_location1 = ws['A3:D11']  # Assuming data for location 1 is in column A starting from row 2
data_location1 = [[cell.value for cell in row] for row in data_range_location1]

data_range_location2 = ws['A12:D18']  # Assuming data for location 2 is in column E starting from row 2
data_location2 = [[cell.value for cell in row] for row in data_range_location2]

# Create separate DataFrames for location 1 and location 2
df_location1 = pd.DataFrame(data_location1, columns=['EmployeeID', 'Data1', 'Data2', 'Data3'])
df_location2 = pd.DataFrame(data_location2, columns=['EmployeeID', 'Data1', 'Data2', 'Data3'])

# Consolidate data based on EmployeeID and sum the values for each location
consolidated_data_location1 = df_location1.groupby('EmployeeID', as_index=False).sum()
consolidated_data_location2 = df_location2.groupby('EmployeeID', as_index=False).sum()

# Merge the consolidated data for employees working at both locations (outer merge)
consolidated_data_both_locations = pd.merge(consolidated_data_location1, consolidated_data_location2, on='EmployeeID', suffixes=('_loc1', '_loc2'), how='inner')

# Calculate the final values for the Data1, Data2, Data3 columns (sum for all)
consolidated_data_both_locations['Data1'] = consolidated_data_both_locations['Data1_loc1'] + consolidated_data_both_locations['Data1_loc2']
consolidated_data_both_locations['Data2'] = consolidated_data_both_locations['Data2_loc1'] + consolidated_data_both_locations['Data2_loc2']
consolidated_data_both_locations['Data3'] = consolidated_data_both_locations['Data3_loc1'] + consolidated_data_both_locations['Data3_loc2']

# Remove the unnecessary columns from the consolidated DataFrame
consolidated_data_both_locations.drop(columns=['Data1_loc1', 'Data1_loc2', 'Data2_loc1', 'Data2_loc2', 'Data3_loc1', 'Data3_loc2'], inplace=True)

# Convert the consolidated data to a list
consolidated_data_list = consolidated_data_both_locations.values.tolist()

# Determine the starting row for placing the consolidated data
start_row = 20  # Assuming you want to start placing the consolidated data in row 20

# Put the consolidated data back into the Excel worksheet
for row_index, row_data in enumerate(consolidated_data_list):
    for col_index, cell_value in enumerate(row_data):
        cell = ws.cell(row=start_row + row_index, column=col_index + 1)
        cell.value = cell_value

# Save the updated workbook
wb.save("StaffPerformanceOverviewtest-06-23 test copy.xlsx")


***

***

In [19]:
import pandas as pd
from openpyxl import load_workbook

# Load the existing Excel workbook
wb = load_workbook("StaffPerformanceOverviewtest-06-23 test copy.xlsx")

# Select the worksheet you want to update (assuming it's the first sheet)
ws = wb.active

# Read the data from the worksheet into a Pandas DataFrame
data_range_location1 = ws['A3:D10']  # Assuming data for location 1 is in column A starting from row 2
data_location1 = [[cell.value for cell in row] for row in data_range_location1]

data_range_location2 = ws['A11:D18']  # Assuming data for location 2 is in column E starting from row 2
data_location2 = [[cell.value for cell in row] for row in data_range_location2]

# Create separate DataFrames for location 1 and location 2
df_location1 = pd.DataFrame(data_location1, columns=['EmployeeID', 'Data1', 'Data2', 'Data3'])
df_location2 = pd.DataFrame(data_location2, columns=['EmployeeID', 'Data1', 'Data2', 'Data3'])

# Consolidate data based on EmployeeID and sum the values for each location
consolidated_data_location1 = df_location1.groupby('EmployeeID', as_index=False).sum()
consolidated_data_location2 = df_location2.groupby('EmployeeID', as_index=False).sum()

# Merge the consolidated data for employees working at both locations
consolidated_data_both_locations = pd.merge(consolidated_data_location1, consolidated_data_location2, on='EmployeeID', suffixes=('_loc1', '_loc2'), how='inner')
consolidated_data_both_locations['Data1'] = consolidated_data_both_locations['Data1_loc1'] + consolidated_data_both_locations['Data1_loc2']
consolidated_data_both_locations['Data2'] = consolidated_data_both_locations['Data2_loc1'] + consolidated_data_both_locations['Data2_loc2']
consolidated_data_both_locations['Data3'] = consolidated_data_both_locations['Data3_loc1'] + consolidated_data_both_locations['Data3_loc2']

# Remove the unnecessary columns from the consolidated DataFrame
consolidated_data_both_locations.drop(columns=['Data1_loc1', 'Data1_loc2', 'Data2_loc1', 'Data2_loc2', 'Data3_loc1', 'Data3_loc2'], inplace=True)

# Convert the consolidated data to a list
consolidated_data_list = consolidated_data_both_locations.values.tolist()

# Determine the starting row for placing the consolidated data
start_row = 3  # Assuming you want to start placing the consolidated data in row 2

# Put the consolidated data back into the Excel worksheet
for row_index, row_data in enumerate(consolidated_data_list):
    for col_index, cell_value in enumerate(row_data):
        cell = ws.cell(row=start_row + row_index, column=col_index + 1)
        cell.value = cell_value

# Save the updated workbook
wb.save("StaffPerformanceOverviewtest-06-23 test copy.xlsx")
